<a href="https://colab.research.google.com/github/mariamhergnyan/Molecular_Simulations_exam/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run these before working.

In [32]:
from google.colab import output
output.disable_custom_widget_manager()

In [21]:
!pip install pyemma > /dev/null
!pip install mdshare > /dev/null

In [22]:
! pip install nglview > /dev/null

ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [23]:
! pip install biopython > /dev/null

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma
# for visualization of molecular structures:
import nglview as nv
import mdtraj
from threading import Timer
from nglview.player import TrajectoryPlayer
from Bio.PDB import PDBParser

In [ ]:
#mdshare.fetch is used for fetching from their server.
#In my case I have it locally, so it just should be the path

In [31]:
pdb = '/content/lb6-filtered.pdb'
files = '/content/lb6-filtered.xtc'

# Create an NGLview widget to display the PDB file
view = nv.show_file(pdb)
view.add_representation('cartoon')
view.center()
view


NGLWidget()

In [18]:
# Parse the PDB file using BioPython's PDBParser
parser = PDBParser()
structure = parser.get_structure('my_structure', '/content/lb6-filtered.pdb')

# Prepare coordinates for PyEmma
coordinates = []
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                coordinates.append(atom.get_coord())

# Convert coordinates to PyEmma's format (assuming 3D coordinates)
coordinates = [list(atom_coord) for atom_coord in coordinates]


# Perform PyEmma analysis (example: PCA)
#pca = pyemma.coordinates.pca(coordinates)


In [20]:
# Define the file path for saving the coordinates
txt_file = '/content/coordinates.txt'


# Write coordinates to a text file
with open(txt_file, 'w') as file:
    for coord in coordinates:
        file.write(','.join(map(str, coord)) + '\n')


In [34]:
torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True, periodic=False)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)
labels = ['backbone\ntorsions']

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Backbone())
positions_data = pyemma.coordinates.load(files, features=positions_feat)
labels += ['backbone atom\npositions']

distances_feat = pyemma.coordinates.featurizer(pdb)
distances_feat.add_distances(
    distances_feat.pairs(distances_feat.select_Backbone(), excluded_neighbors=2), periodic=False)
distances_data = pyemma.coordinates.load(files, features=distances_feat)
labels += ['backbone atom\ndistances']